<a href="https://colab.research.google.com/github/Lchris22/Covid-detection-FL/blob/colab/colab_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q flwr["simulation"] tensorflow

import math
import os

# Make TensorFlow logs less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf

import flwr as fl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.5 

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_val, y_val) -> None:
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        r=self.model.fit(self.x_train, self.y_train, epochs=5, verbose=2)
        hist = r.history
        print("Fit history : " ,hist)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, acc = self.model.evaluate(self.x_val, self.y_val, verbose=2)
        return loss, len(self.x_val), {"accuracy": acc}


In [ ]:
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

CLIENT_1_DATA_DIR = "/content/drive/MyDrive/dataset_split/client1"
CLIENT_2_DATA_DIR = "/content/drive/MyDrive/dataset_split/client2"

# Load dataset for client 1
train_dir1 = CLIENT_1_DATA_DIR+"/train"
test_dir1 = CLIENT_1_DATA_DIR+"/test"

train_dir2 = CLIENT_2_DATA_DIR+"/train"
test_dir2 = CLIENT_2_DATA_DIR+"/test"

def client_fn(cid: str) -> fl.client.Client:
   # Load and compile Keras model
    vgg = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    # Freeze first 10 layers
    for layer in vgg.layers[:10]:
        layer.trainable = False
    x = vgg.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)  # change number of classes to 2 for covid and normal
    model = Model(inputs=vgg.input, outputs=predictions)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

     # Load training and validation data for client
    if cid == "0":
        train_dir = train_dir1
        test_dir = test_dir1
    elif cid == "1":
        train_dir = train_dir2
        test_dir = test_dir2
    else:
        raise ValueError(f"Invalid client ID {cid}")
    print("----------------------------ID is ",cid,"------------------------------------")

    train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator1 = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), class_mode='sparse')
    test_generator1 = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), class_mode='sparse')

    # Extract x_train1, y_train1, x_val1, y_val1 from train_generator1 and test_generator1
    # using the `next()` function.
    x_train1, y_train1 = next(train_generator1)
    x_val1, y_val1 = next(test_generator1)

    return FlowerClient(model, x_train1, y_train1, x_val1, y_val1)

In [ ]:
import numpy as np
NUM_CLIENTS = 2

class SaveModelStrategy(fl.server.strategy.FedAvg):
    def aggregate_fit(
        self,
        rnd,
        results,
        failures
    ):
      aggregated_weights = super().aggregate_fit(rnd, results, failures)
      if aggregated_weights is not None:
          # Save aggregated_weights
          print(f"Saving round {rnd} aggregated_weights...")
          np.savez(f"round-{rnd}-weights.npz", *aggregated_weights)
      return aggregated_weights

strategy = SaveModelStrategy()

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=20),
    strategy=strategy,
)

INFO flwr 2023-04-28 10:49:19,524 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)
2023-04-28 10:49:24,272	INFO worker.py:1625 -- Started a local Ray instance.
INFO flwr 2023-04-28 10:49:27,813 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 3872972390.0, 'CPU': 2.0, 'node:172.28.0.12': 1.0, 'memory': 7745944782.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 3872972390.0, 'CPU': 2.0, 'node:172.28.0.12': 1.0, 'memory': 7745944782.0}
INFO flwr 2023-04-28 10:49:27,819 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-04-28 10:49:27,824 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client


(launch_and_get_parameters pid=3180) ----------------------------ID is  1 ------------------------------------
(launch_and_get_parameters pid=3180) Found 3000 images belonging to 2 classes.
(launch_and_get_parameters pid=3180) Found 616 images belonging to 2 classes.


INFO flwr 2023-04-28 10:50:31,095 | server.py:277 | Received initial parameters from one random client
INFO:flwr:Received initial parameters from one random client
INFO flwr 2023-04-28 10:50:31,107 | server.py:88 | Evaluating initial parameters
INFO:flwr:Evaluating initial parameters
INFO flwr 2023-04-28 10:50:31,112 | server.py:101 | FL starting
INFO:flwr:FL starting
DEBUG flwr 2023-04-28 10:50:31,118 | server.py:218 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG:flwr:fit_round 1: strategy sampled 2 clients (out of 2)


(launch_and_fit pid=3180) ----------------------------ID is  1 ------------------------------------
(launch_and_fit pid=3180) Found 3000 images belonging to 2 classes.
(launch_and_fit pid=3180) Found 616 images belonging to 2 classes.
(launch_and_fit pid=3181) ----------------------------ID is  0 ------------------------------------
(launch_and_fit pid=3181) Found 3000 images belonging to 2 classes.
(launch_and_fit pid=3181) Found 616 images belonging to 2 classes.
(launch_and_fit pid=3180) Epoch 1/5
(launch_and_fit pid=3181) Epoch 1/5
(launch_and_fit pid=3180) 1/1 - 102s - loss: 0.7185 - accuracy: 0.3750 - 102s/epoch - 102s/step
(launch_and_fit pid=3180) Epoch 2/5
(launch_and_fit pid=3181) 1/1 - 132s - loss: 0.7223 - accuracy: 0.3438 - 132s/epoch - 132s/step
(launch_and_fit pid=3181) Epoch 2/5
(launch_and_fit pid=3180) 1/1 - 124s - loss: 9.9824 - accuracy: 0.5625 - 124s/epoch - 124s/step
(launch_and_fit pid=3180) Epoch 3/5
(launch_and_fit pid=3181) Epoch 3/5
(launch_and_fit pid=3181) 

DEBUG flwr 2023-04-28 11:01:35,915 | server.py:232 | fit_round 1 received 2 results and 0 failures
DEBUG:flwr:fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-04-28 11:01:36,380 | fedavg.py:243 | No fit_metrics_aggregation_fn provided


...

Saving round 20 aggregated_weights...


DEBUG flwr 2023-04-28 14:41:19,712 | server.py:168 | evaluate_round 20: strategy sampled 2 clients (out of 2)
DEBUG:flwr:evaluate_round 20: strategy sampled 2 clients (out of 2)


(launch_and_evaluate pid=3180) ----------------------------ID is  0 ------------------------------------
(launch_and_evaluate pid=3181) Found 3000 images belonging to 2 classes.
(launch_and_evaluate pid=3181) Found 616 images belonging to 2 classes.
(launch_and_evaluate pid=3181) 1/1 - 51s - loss: 0.6915 - accuracy: 0.6250 - 51s/epoch - 51s/step
(launch_and_evaluate pid=3181) 1/1 - 51s - loss: 0.6915 - accuracy: 0.6250 - 51s/epoch - 51s/step
(launch_and_evaluate pid=3180) Found 616 images belonging to 2 classes. [repeated 2x across cluster]


DEBUG flwr 2023-04-28 14:42:25,388 | server.py:182 | evaluate_round 20 received 2 results and 0 failures
DEBUG:flwr:evaluate_round 20 received 2 results and 0 failures
INFO flwr 2023-04-28 14:42:25,391 | server.py:147 | FL finished in 13914.273776025999
INFO:flwr:FL finished in 13914.273776025999
INFO flwr 2023-04-28 14:42:25,413 | app.py:218 | app_fit: losses_distributed [(1, 0.716153621673584), (2, 0.8664299845695496), (3, 0.6774159073829651), (4, 0.6892070770263672), (5, 0.6919582486152649), (6, 0.6932765245437622), (7, 0.6943458318710327), (8, 0.6971553266048431), (9, 0.6932258605957031), (10, 0.6928055584430695), (11, 0.6901353001594543), (12, 0.7150063514709473), (13, 0.7018144428730011), (14, 0.694442093372345), (15, 0.6970476508140564), (16, 0.6869297623634338), (17, 0.6943231225013733), (18, 0.6945726275444031), (19, 0.6931465864181519), (20, 0.6923280358314514)]
INFO:flwr:app_fit: losses_distributed [(1, 0.716153621673584), (2, 0.8664299845695496), (3, 0.6774159073829651), (4

History (loss, distributed):
	round 1: 0.716153621673584
	round 2: 0.8664299845695496
	round 3: 0.6774159073829651
	round 4: 0.6892070770263672
	round 5: 0.6919582486152649
	round 6: 0.6932765245437622
	round 7: 0.6943458318710327
	round 8: 0.6971553266048431
	round 9: 0.6932258605957031
	round 10: 0.6928055584430695
	round 11: 0.6901353001594543
	round 12: 0.7150063514709473
	round 13: 0.7018144428730011
	round 14: 0.694442093372345
	round 15: 0.6970476508140564
	round 16: 0.6869297623634338
	round 17: 0.6943231225013733
	round 18: 0.6945726275444031
	round 19: 0.6931465864181519
	round 20: 0.6923280358314514